In [1]:
import numpy as np
import cv2
import time
import glob

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import dv

from joblib import dump, load

In [2]:
def loadLabel(path):
    labels = np.genfromtxt(path, delimiter=',', skip_header=1, dtype=int)

    start_time = labels[0][1]
    for i in labels:
        i[1] = i[1]-start_time
        i[2] = i[2]-start_time
        
    return labels

In [3]:
# Return the start time of the given class index.
def start(index, labels):
    return labels[index][1]
 
# Return the end time of the given class index.
def end(index, labels):
    return labels[index][2]

def classID(index, labels):
    return labels[index][0]-1

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime, labels):
    for i in range(len(labels)-1):
        if (currentTime >= start(i, labels) and currentTime <= end(i, labels)):
            return classID(i, labels)
    return -1

In [4]:
def verify(csv, file):
    
    csv = csv.split("_labels")
    file = file.split(".")
    
    if (csv[0] != file[0]):
        print("mismatch: (%s, %s)" % (csv[0], file[0]))

In [5]:
csv = sorted(glob.glob("data/*.csv"))
aedat4 = sorted(glob.glob("data/*.aedat4"))

In [6]:
pipeline = load('pipeline.joblib')

In [7]:
def process(X, Y, path, labels):
    offset = 0
    
    with dv.AedatFile(path) as f: 
        for frame in f['frames']:    
            # Set the offset on the first frame.
            if (offset == 0):
                offset = frame.timestamp

            gesture = setGesture((frame.timestamp - offset), labels)
            image = frame.image

            # font
            font = cv2.FONT_HERSHEY_SIMPLEX

            # org
            org = (5, 30)

            # fontScale
            fontScale = 1

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 2
            image = cv2.putText(image, str(gesture), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
            
            # cv2.imshow('out',image)
            # cv2.waitKey(1)

            if (gesture == -1):
                continue
            else:
                Y.append(gesture)

                # TODO: Scale the data.
                matrix = frame.image.flatten()
                X.append(matrix)

In [8]:
sample = 114
X_test = []
Y_test = []

verify(csv[sample], aedat4[sample])
print(csv[sample], aedat4[sample])

labels = loadLabel(csv[sample])
process(X_test, Y_test, aedat4[sample], labels)

data\user28_lab_labels.csv data\user28_lab.aedat4


pipeline.score(X_test, Y_test)

In [9]:
n=50

start_time = time.time()
for i in range(n):
    pipeline.predict(X_test[i].reshape(1,-1))[0]

print(time.time()-start_time)
print((time.time()-start_time)/n)

51.57305669784546
1.031461133956909


In [10]:
def viewer(path, labels):
    offset = 0
    
    with dv.AedatFile(path) as f: 
        for frame in f['frames']:    
            # Set the offset on the first frame.
            if (offset == 0):
                offset = frame.timestamp

            gesture = setGesture((frame.timestamp - offset), labels)
            image = frame.image

            # font
            font = cv2.FONT_HERSHEY_SIMPLEX

            # org
            org = (5, 30)

            # fontScale
            fontScale = 1

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 2
            image = cv2.putText(image, str(gesture), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)

            matrix = frame.image.flatten()
            predicted = pipeline.predict(matrix.reshape(1,-1))[0]
            
            org = (240, 30)
            
            if (predicted == gesture):
                color = (255, 0, 0)
            else:
                color = (100, 0, 0)
            
            image = cv2.putText(image, str(predicted), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
                
            cv2.imshow('out',image)
            cv2.waitKey(1)

viewer(aedat4[sample], labels)